In [8]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [9]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as pyplot
import optuna as op
%matplotlib inline
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Evaluation.K_Fold_Evaluator import K_Fold_Evaluator_MAP
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Utils.createURM import createURM
from Utils.createICM import createICM
from Utils.combineURMICM import combine

### URM

In [10]:
URM = createURM()
ICM = createICM()

C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Utils\createURM.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('../Input/interactions_and_impressions.csv')
C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Utils\createICM.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  URM = pd.read_csv('../Input/interactions_and_impressions.csv')


In [11]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

In [12]:
evaluator_validation = K_Fold_Evaluator_MAP(URM_validation, cutoff_list=[10], verbose=False)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [ ]:
URMICMCombined = combine(ICM=ICM, URM = URM)

### SLIM

In [6]:
recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URMICMCombined.T, verbose=False)
#recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train, verbose=False)

def objective(trial):
    alpha = trial.suggest_float("alpha", 0, 1)
    l1_ratio = trial.suggest_float("l1_ratio", 0, 1)
    topK = trial.suggest_float("topK", 100, 150)
    recommender.fit(alpha=alpha, l1_ratio=l1_ratio, topK=int(topK))
    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    return result_dict.loc[10]["MAP"]

In [13]:
study = op.create_study(direction='minimize')
study.optimize(objective, n_trials=3)

[I 2022-11-04 19:38:24,249] A new study created in memory with name: no-name-ceea85e5-897e-4675-9be3-a2def6158953

100%|█████████▉| 24504/24507 [05:32<00:00, 73.69it/s]


EvaluatorHoldout: Processed 38904 (100.0%) in 18.87 sec. Users per second: 2062


[I 2022-11-04 19:44:15,698] Trial 0 finished with value: 1.9186422255515413e-05 and parameters: {'alpha': 0.26609508940760473, 'l1_ratio': 0.2514101322413186, 'topK': 282.20864536824354}. Best is trial 0 with value: 1.9186422255515413e-05.

100%|█████████▉| 24504/24507 [05:44<00:00, 71.08it/s]


EvaluatorHoldout: Processed 38904 (100.0%) in 19.02 sec. Users per second: 2045


[I 2022-11-04 19:50:19,528] Trial 1 finished with value: 1.9186422255515413e-05 and parameters: {'alpha': 0.5618682386028638, 'l1_ratio': 0.707654698860444, 'topK': 173.42278484032772}. Best is trial 0 with value: 1.9186422255515413e-05.

100%|█████████▉| 24504/24507 [05:39<00:00, 72.25it/s]


EvaluatorHoldout: Processed 38904 (100.0%) in 19.02 sec. Users per second: 2045


[I 2022-11-04 19:56:17,786] Trial 2 finished with value: 0.0014294904592174929 and parameters: {'alpha': 0.7548063350432871, 'l1_ratio': 0.02309817497809641, 'topK': 295.84345878082394}. Best is trial 0 with value: 1.9186422255515413e-05.


In [14]:
alpha = study.best_params['alpha']
l1_ratio = study.best_params['l1_ratio']
topK = study.best_params['topK']

In [15]:
alpha

0.26609508940760473

In [16]:
l1_ratio

0.2514101322413186

In [17]:
topK

282.20864536824354

In [18]:
recommender.fit(alpha=0.8804905330217533, l1_ratio=0.9120820630476552, topK=int(491.2852695850108))
result_df, _ = evaluator_test.evaluateRecommender(recommender)
result_df


100%|█████████▉| 24504/24507 [05:36<00:00, 72.74it/s]


EvaluatorHoldout: Processed 40172 (100.0%) in 19.37 sec. Users per second: 2074


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.000092,0.000171,0.000165,0.000032,0.000061,0.000319,0.000145,0.000118,0.000921,0.000319,...,0.965,0.000889,0.965,0.000502,3.79044,0.91986,0.001153,0.287603,0.04466,0.738285


In [13]:
recommender.fit(alpha=0.7144444121186274, l1_ratio=0.8941505797892362, topK=int(193.62943556987977))
result_df, _ = evaluator_test.evaluateRecommender(recommender)
result_df


100%|█████████▉| 24504/24507 [01:11<00:00, 342.34it/s]


EvaluatorHoldout: Processed 40109 (100.0%) in 13.47 sec. Users per second: 2977


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.963487,0.0,0.963487,0.000591,4.095323,0.928331,0.001274,0.308715,0.047869,0.450896
